In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from plotnine import ggplot,aes
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df=pd.read_csv("/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv")
print(df.shape)
df.head()

In [ ]:
df.describe().round(3)

In [ ]:
df.dtypes

In [ ]:
print(df.isna().sum())

Analizo la correlacion de todas las variables con Rainfall

In [ ]:
corr_matrix=df.corr()
corr_matrix['MaxTemp'].sort_values(ascending=False)

# Exploratory Data Analysis (EDA)

## Variables Continuas

In [ ]:
fig=plt.figure(figsize=(20,12),facecolor='white')

plt.suptitle('Univariate Distributions',weight='bold',fontsize='xx-large')

gs=fig.add_gridspec(2,2)

ax=[_ for i in range(4)]

ax[0]=fig.add_subplot(gs[0,0])
ax[1]=fig.add_subplot(gs[0,1])
ax[2]=fig.add_subplot(gs[1,0])
ax[3]=fig.add_subplot(gs[1,1])


sns.histplot(x="Temp3pm",data=df,ax=ax[0],shrink=0.5,bins=50,color="lightcoral")
sns.histplot(x="Humidity3pm",data=df,ax=ax[3],bins=20,color='darkred')
sns.boxplot(x='Cloud3pm',data=df,ax=ax[1],color='gold')
sns.violinplot(x="Sunshine",data=df,ax=ax[2],color="greenyellow")

for i in range(4):
    
    ax[i].set_yticklabels('')
    ax[i].set_ylabel('')
    ax[i].tick_params('y',length=0)
    
    for direction in ['top','left','right']:
        
        ax[i].spines[direction].set_visible(False)

In [ ]:
ax_0=sns.lineplot(data=df,x='Sunshine',y='Evaporation',hue='RainTomorrow')

In [ ]:
ax_1=sns.scatterplot(data=df,x='Cloud3pm',y='Rainfall',hue='RainTomorrow',palette='viridis',markers='o',alpha=0.1)

## Variables Categoricas

In [ ]:
var_cat = df.select_dtypes(include='object').columns #these are the categorical columns
var_cat

In [ ]:
for x in var_cat:  
    print(x+"  "+str(df[x].nunique()))

In [ ]:
sns.catplot(x="RainToday", y="Temp3pm", col='RainTomorrow', data=df, kind = "bar");

# Regresion Lineal Simple (RLS)

In [ ]:
ax=df.plot(kind='scatter', x='Temp3pm', y='MaxTemp', alpha=0.2, figsize=(9,4))

In [ ]:
df.dropna(subset=['MaxTemp', 'Temp3pm'], inplace=True)
print(df.isna().sum().sort_values(ascending=True))

In [ ]:
#variable predictora
x=df.Temp3pm.to_numpy()
x= x.reshape([-1,1])
#Lavels (valores reales)/ Targets
t=df.MaxTemp.to_numpy()

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

## Veo que tan similares son los histogramas de las dos variables

In [ ]:
plt.hist(y, histtype='step', label='predicción')
plt.hist(t, histtype='step', label='target')
plt.legend(loc='upper left')

In [ ]:
lr.fit(x, t)
y= lr.predict(x)

# Calculo R^2 o coheficiente de determinacion

In [ ]:
r2=  mean_squared_error(y, t)
print('El R^2 del modelo lineal es {:.3f}'.format(r2))

# Calculo RMSE / R

In [ ]:
rmse = np.sqrt(np.sum((y - t)**2)/len(y))
print('El RMSE del modelo lineal es {:.3f}'.format(rmse))
#RMSE = R

In [ ]:
plt.plot(x,t,'.b', alpha=1)
plt.plot(x,y,'or')
plt.xlabel('x= Temp3pm')
plt.ylabel('y= MaxTemp')

## $\omega_0$ y $\omega_1$ / parámetros de mi modelo :

In [ ]:
print("Wo={} y W1={}".format(lr.coef_,lr.intercept_))

# Regresion Lineal Multiple (RLM)

La regresión lineal múltiple permite generar un modelo lineal en el que el valor de la variable dependiente o respuesta (Y) se determina a partir de un conjunto de variables independientes llamadas predictores (X1, X2, X3…).